In [9]:
import psycopg2
import datetime
import numpy as np
import pandas as pd
from itertools import islice

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn import preprocessing
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters

from finance.utilities import utils, financial_utils, time_series_utils, cluster_utils, random_forest_utils

ModuleNotFoundError: No module named 'finance'

In [7]:
import os
os.getcwd()

'/usr/src/app/finance/data_science/exploration'

In [8]:
os.listdir()

['stock_explorer.ipynb',
 'gpu_explorer.ipynb',
 'benchmark_explorer.ipynb',
 '.ipynb_checkpoints',
 'options_explorer.ipynb',
 'nlp_explorer.ipynb']

In [2]:
register_matplotlib_converters()

%matplotlib notebook
%matplotlib inline

NameError: name 'register_matplotlib_converters' is not defined

### Stock data

In [3]:
query = """
    select
        e.symbol
        , e.market_datetime
        , e.open
        , e.high
        , e.low
        , e.close
        , e.volume
    from td.equities as e
    where left(e.symbol, 1) = 'A'
    order by e.market_datetime
    limit 100
    """

df = utils.query_db(db_connection=utils.DW_STOCKS_JUPYTER, query=query)
df.head()

NameError: name 'utils' is not defined

### Check for gpus

In [6]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(1))

True
GeForce GTX 1080 Ti


In [5]:
df = cluster_utils.encode_one_hot(df=df, column='symbol')

In [6]:
for col in ['open', 'high', 'low', 'close', 'volume']:
    df = cluster_utils.normalize(df=df, column=col, subset='symbol')

In [7]:
train = df.copy()
train = train.drop(columns=['symbol'])
train['market_datetime'] = train['market_datetime'].astype(int)

In [8]:
train_target = train['open'].shift(-1)

In [9]:
split_size = 80000
train_x = train.iloc[1:split_size].values
train_y = train_target.iloc[1:split_size].values

test_x = train.iloc[split_size+1:-1].values
test_y = train_target.iloc[split_size+1:-1].values

In [12]:
print(torch.cuda.memory_allocated())

print(torch.cuda.memory_cached())

70144
2097152


In [11]:
tensor_x = torch.tensor(train_x).cuda(0)
tensor_y = torch.tensor(train_y).cuda(0)

In [13]:
input_size = 87
output_size = 1
learning_rate = 0.01 
epochs = 100

model = torch.nn.Linear(input_size, output_size).cuda(0)

criterion = torch.nn.MSELoss().cuda(0)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [14]:
with torch.cuda.device(1):
    for t in range(2):
        prediction = model(tensor_x.float()).cuda(0)

        loss = criterion(prediction, tensor_y.float()).cuda(0)

        optimizer.zero_grad() #.cuda(0)
        loss.backward() #.cuda(0)
        optimizer.step() #.cuda(0)
        
        loss.item()

/home/nautilus/anaconda3/envs/babylon/lib/python3.6/site-packages/torch/nn/modules/loss.py:443: UserWarning: Using a target size (torch.Size([99])) that is different to the input size (torch.Size([99, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/nautilus/anaconda3/envs/babylon/lib/python3.6/site-packages/torch/nn/modules/loss.py:443: UserWarning: Using a target size (torch.Size([99])) that is different to the input size (torch.Size([99, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [15]:
loss.item()

nan